In [117]:
# 1.目的
# 予想脚質がゴール直近の通貨順位に対してどのくらい正確なのか調べたい
# （正確なのであれば、例えば予想脚質が"逃げ"ならゴール直前の通貨順位は小さいはず）
# 2.やったこと
# XGBoostを使う。目的変数を通過順位とし、説明変数を予想脚質として予測したときの精度を調べる
# ただし、学習データは同レースにおける馬の脚質予想の割合でフィルタリングしてから分析に用いる。
# なぜならば、脚質予想の割合が異なるレースが混じっている状態で正しく分析できないからである。
# 例えば通過順位が3位であるとしても、以下の2パターンが考えられる。
# ・予想脚質が追込である馬が8頭中6頭いて、追込の馬が後方集団にいながら通過順位が3位である
# ・予想脚質が逃げである馬が8頭中3頭いて、逃げの馬が前方集団にいながら通過順位が3位である
# 3.結果に対する所感
# 予想脚質は通過順位に有意であると考えられる。
# 特に逃げ・先行の頭数と差し・追込の頭数の差が小さいほど有意であると考えられる。

In [7]:
# インポート
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score

In [113]:
# CSVファイルの読み込み
# df = pd.read_csv('input3.csv', encoding='cp932')
df = pd.read_csv('input3.csv', encoding='utf-8')
# style_nameはカテゴリ変数であり文字列だがそのまま使えないのでエンコーディングして数値に変換する
# 脚質を one-hot エンコーディングして、レースIDごとに集計
style_counts = pd.get_dummies(df['style_name']).groupby(df['race_id']).sum()
# 列名をわかりやすく変更
style_counts = style_counts.rename(columns={
    '逃げ': '逃げ頭数',
    '先行': '先行頭数',
    '差し': '差し頭数',
    '追込': '追込頭数'
})
# 元の df にマージ（レースIDに基づいて）
df = df.merge(style_counts, left_on='race_id', right_index=True)
# 逃げ + 先行 の合計カラムを追加
df['逃げ・先行頭数'] = df['逃げ頭数'] + df['先行頭数']
# 差し + 追込 の合計カラムを追加
df['差し・追込頭数'] = df['差し頭数'] + df['追込頭数']
# 結果を確認
print(df.head())
# 明示的にマッピングする辞書を定義
style_mapping = {
    '逃げ': 1,
    '先行': 2,
    '差し': 3,
    '追込': 4
}
# 新しい列 'style_encoded' を追加（元の 'style_name' 列はそのまま）
df['style_encoded'] = df['style_name'].map(style_mapping)
# 頭数の割合でフィルタリング
number_of_FR = 3
number_of_RR = 5
number_of_style_filtered_df = df[(df['逃げ・先行頭数'] == number_of_FR) & (df['差し・追込頭数'] == number_of_RR)]
print(f"フィルタリング後のレコード数: {len(number_of_style_filtered_df)}")
# レースIDの先頭2桁を抽出（文字列として扱う）
number_of_style_filtered_df.loc[:, 'race_id_year'] = number_of_style_filtered_df['race_id'].astype(str).str[:2]
# 対象：23 または 24 のレースのみ抽出
year_filtered_df = number_of_style_filtered_df[number_of_style_filtered_df['race_id_year'].isin(['23', '24'])]
# 件数カウント
count_23 = (year_filtered_df['race_id_year'] == '23').sum()
count_24 = (year_filtered_df['race_id_year'] == '24').sum()
# 割合を計算
rate_24 = count_24 / (count_23 + count_24)
# 結果表示
print(f"24の割合: {rate_24:.2f}")
# 回帰の場合→XGBRegressorを使う
# 説明変数
x = number_of_style_filtered_df[["style_encoded"]]
# 目的変数
y1 = number_of_style_filtered_df["position_1"]
y2 = number_of_style_filtered_df["position_2"]
y3 = number_of_style_filtered_df["position_3"]
# 学習データと検証データの分割（検証データ割合はrate_24）
x_train, x_test, y1_train, y1_test = train_test_split(x, y1, train_size=rate_24, shuffle=False)
x_train, x_test, y2_train, y2_test = train_test_split(x, y2, train_size=rate_24, shuffle=False)
x_train, x_test, y3_train, y3_test = train_test_split(x, y3, train_size=rate_24, shuffle=False)
# モデルの学習
model1 = xgb.XGBRegressor(objective='reg:squarederror', max_depth=3, random_state=42)
model1.fit(x_train, y1_train)
model2 = xgb.XGBRegressor(objective='reg:squarederror', max_depth=3, random_state=42)
model2.fit(x_train, y2_train)
model3 = xgb.XGBRegressor(objective='reg:squarederror', max_depth=3, random_state=42)
model3.fit(x_train, y3_train)
# 予測
y1_pred = model1.predict(x_test)
y2_pred = model2.predict(x_test)
y3_pred = model3.predict(x_test)
# 評価
rmse1 = np.sqrt(mean_squared_error(y1_test, y1_pred))
r2_1 = r2_score(y1_test, y1_pred)
print(f"RMSE: {rmse1:.4f}")
print(f"R^2 Score: {r2_1:.4f}")
rmse2 = np.sqrt(mean_squared_error(y2_test, y2_pred))
r2_2 = r2_score(y2_test, y2_pred)
print(f"RMSE: {rmse2:.4f}")
print(f"R^2 Score: {r2_2:.4f}")
rmse3 = np.sqrt(mean_squared_error(y3_test, y3_pred))
r2_3 = r2_score(y3_test, y3_pred)
print(f"RMSE: {rmse3:.4f}")
print(f"R^2 Score: {r2_3:.4f}")

     race_id  horse_number       race       horse jockey  frame_number  style  \
0  230101C01             1  230101C01  2019104863   5615             1    3.0   
1  230101C01             2  230101C01  2009105463  a022d             2    4.0   
2  230101C01             3  230101C01  2019100160   5299             3    3.0   
3  230101C01             4  230101C01  2019110030   5610             4    3.0   
4  230101C01             5  230101C01  2019103820  a0258             5    1.0   

    odds  popularity  finish_rank  ...  jockey_place_rate_7  \
0    8.4           4            4  ...                0.233   
1  319.4           8            6  ...                0.000   
2    1.8           1            1  ...                0.567   
3   38.1           6            3  ...                0.367   
4    3.9           3            8  ...                0.400   

   jockey_place_rate_8  jockey_place_rate_9  conditional_place_rate_10  先行頭数  \
0                0.225                 0.20           

C:\Users\ryo\AppData\Local\Temp\ipykernel_5296\546242745.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  number_of_style_filtered_df.loc[:, 'race_id_year'] = number_of_style_filtered_df['race_id'].astype(str).str[:2]
